Notebook to handle with the use of OceanParcels to run a simulation with different setting of parameters

In [16]:
import xarray as xr
import pandas as pd
import numpy as np
from datetime import timedelta
from parcels import FieldSet, Field, ParticleSet, Variable, JITParticle, AdvectionRK4, Kernel

In [17]:
# This passage only to see in detail how the three datasets are made, unnecessary for the fieldset step
days100_from_april_1997_velocity_u_OK = xr.open_dataset("/mnt/iscsi2/OceanParcels/InputObsDom/days100_from_april_1997_velocity_u_OK.nc")
days100_from_april_1997_velocity_v_OK = xr.open_dataset("/mnt/iscsi2/OceanParcels/InputObsDom/days100_from_april_1997_velocity_v_OK.nc")

### FieldSet

NOTE: assign the correct path of the .nc files. Here in this way because they are in the same folder

In [18]:
ufiles = "/mnt/iscsi2/OceanParcels/InputObsDom/days100_from_april_1997_velocity_u_OK.nc"
vfiles = "/mnt/iscsi2/OceanParcels/InputObsDom/days100_from_april_1997_velocity_v_OK.nc"
mesh_mask = "/mnt/iscsi2/OceanParcels/InputObsDom/WINDS_as_NEMO_mesh_prof.nc"
coords = xr.open_dataset(mesh_mask, decode_times=False)

### FieldSet

Following "from_nemo" method. It's possible to use different methods.

In [19]:
filenames = {
    "U": {"lon": mesh_mask, "lat": mesh_mask, "data": ufiles},
    "V": {"lon": mesh_mask, "lat": mesh_mask, "data": vfiles},
}

variables = {"U": "u_surf", "V": "v_surf"}

dimensions = {
    "U": {"lon": "lon_psi", "lat": "lat_psi", "time": "time_counter"},
    "V": {"lon": "lon_psi", "lat": "lat_psi", "time": "time_counter"},
}

fieldset = FieldSet.from_nemo(filenames, variables, dimensions, netcdf_decodewarning=False)

### Avoiding errors close to boundaries

To tell Parcels that particles that leave the domain need to be deleted, it's convenient to use a Recovery Kernel, which will be invoked when a particle encounters an ErrorOutOfBounds error, as suggested by Erik van Sebille (https://github.com/OceanParcels/parcels/discussions/1086#discussioncomment-1457212) and in a way similar to this tutorial: https://docs.oceanparcels.org/en/latest/examples/tutorial_kernelloop.html#Working-with-Status-Codes

In [20]:
def DeleteParticle(particle, fieldset, time):
    if particle.state == StatusCode.ErrorOutOfBounds:
        particle.delete()

### Releases initialisation

In [21]:
random_latitudes=np.load('/mnt/iscsi2/OceanParcels/InputObsDom/random_latitude_10250.npy')
random_longitudes=np.load('/mnt/iscsi2/OceanParcels/InputObsDom/random_longitude_10250.npy')

# Number of locations
n_locations = len(random_longitudes)

# Starting time set at 18:00:00 of the 1st of April 1997 (half-hour dataset, 18*2)
start_time = days100_from_april_1997_velocity_u_OK.time_counter[36].values

# Hours of release of the particles (every day, for 29 days a set of particles is released every hour for 12 hours)
release_hours = np.concatenate([np.arange(start, start + 12) for start in np.arange(0, 24*29, 24)])
total_particles_per_location = len(release_hours)
total_particles = n_locations * total_particles_per_location

# Times when a set of particles is released (pay attention to the format of the time,
    # because since this variable will be in the ParticleSet, only this specific format is valid)
release_times = np.tile(release_hours, n_locations) * timedelta(hours=1).total_seconds()
release_times = start_time + release_times.astype('timedelta64[s]')

# All the longitudes and latitudes of releasing
lon = np.repeat(random_longitudes, total_particles_per_location)
lat = np.repeat(random_latitudes, total_particles_per_location)

### ParticleSet

In [22]:
# Create the ParticleSet
pset = ParticleSet(fieldset=fieldset,
                   pclass=JITParticle,
                   lon=lon,
                   lat=lat,
                   time=release_times
                  )

# Create an output file
output_file = pset.ParticleFile(name="/mnt/iscsi2/OceanParcels/resultsObsDom/April97_100days.zarr",
                                outputdt=timedelta(hours=1))

# Execute the particle set for 99 days
pset.execute([AdvectionRK4, DeleteParticle], 
             runtime=timedelta(days=99),  # Total runtime of 99 days
             dt=timedelta(minutes=30),
             output_file=output_file)

  0%|          | 0/8553600.0 [05:34<?, ?it/s]
  0%|          | 0/8553600.0 [02:30<?, ?it/s]
INFO: Output files are stored in /mnt/iscsi2/OceanParcels/resultsObsDom/April98_100days.zarr.
  0%|          | 16200.0/8553600.0 [00:05<47:26, 2999.44it/s]

KeyboardInterrupt: 

  0%|          | 16200.0/8553600.0 [00:17<47:26, 2999.44it/s]

Following "from_nemo" method. It's possible to use different methods.

In [ ]:
filenames = {
    "U": {"lon": mesh_mask, "lat": mesh_mask, "data": ufiles},
    "V": {"lon": mesh_mask, "lat": mesh_mask, "data": vfiles},
}

variables = {"U": "u_surf", "V": "v_surf"}

dimensions = {
    "U": {"lon": "lon_psi", "lat": "lat_psi", "time": "time_counter"},
    "V": {"lon": "lon_psi", "lat": "lat_psi", "time": "time_counter"},
}

fieldset = FieldSet.from_nemo(filenames, variables, dimensions, netcdf_decodewarning=False)

### Avoiding errors close to boundaries

To tell Parcels that particles that leave the domain need to be deleted, it's convenient to use a Recovery Kernel, which will be invoked when a particle encounters an ErrorOutOfBounds error, as suggested by Erik van Sebille (https://github.com/OceanParcels/parcels/discussions/1086#discussioncomment-1457212) and in a way similar to this tutorial: https://docs.oceanparcels.org/en/latest/examples/tutorial_kernelloop.html#Working-with-Status-Codes

In [ ]:
def DeleteParticle(particle, fieldset, time):
    if particle.state == StatusCode.ErrorOutOfBounds:
        particle.delete()